<a href="https://colab.research.google.com/github/22922511/Predict-Customer-Lifetime-Value-/blob/main/Predict%20Customer%20Lifetime%20Value%20(CLV)l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


In [17]:
import pandas as pd
df = pd.read_excel("online_retail_II.xlsx", engine="openpyxl")
df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [18]:
# === 2. Data Cleaning ===
df = df[df['Customer ID'].notna()]
df['Customer ID'] = df['Customer ID'].astype(int)

/tmp/ipython-input-18-2727312771.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Customer ID'] = df['Customer ID'].astype(int)


In [20]:

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df['Total_spend'] = df['Quantity'] * df['Price']


df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M').astype(str)

 agg_data = df.groupby('Customer ID').agg({
    'Total_spend': 'sum',             # Total spend per customer
    'Invoice': 'nunique',             # Number of unique invoices (purchases)
    'Quantity': 'sum',                # Total quantity purchased
    'Price': 'mean',                  # Average price per item
    'Country': 'first'                # Country (assuming it doesn't change per customer)
}).rename(columns={
    'Invoice': 'NumPurchases',
    'Quantity': 'TotalQty',
    'Price': 'AvgPrice'
})

print(agg_data.head())


             Total_spend  NumPurchases  TotalQty   AvgPrice         Country
Customer ID                                                                
12346             -64.68            15        52  12.573043  United Kingdom
12347            1323.32             2       828   2.295070         Iceland
12348             222.16             1       373   0.719500         Finland
12349            2646.99             4       988   8.406449           Italy
12351             300.93             1       261   2.355238     Unspecified


In [21]:

# Encode categorical data (e.g., country)
agg_data = pd.get_dummies(agg_data, columns=['Country'], drop_first=True)

# === 4. Define CLV as target ===
X = agg_data.drop(columns='Total_spend')
y = agg_data['Total_spend']

In [22]:

# === 5. Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
# === 6. Train Model ===
model = GradientBoostingRegressor()
model.fit(X_train, y_train)


GradientBoostingRegressor()

In [24]:
# === 7. Evaluate ===
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 8329.95


In [25]:
# === 8. Predict Future CLV ===
future_clv = model.predict(X)
agg_data['Predicted_CLV'] = future_clv
print(agg_data[['Predicted_CLV']])

             Predicted_CLV
Customer ID               
12346           847.140279
12347          1266.505848
12348           327.331375
12349          2121.393134
12351           411.324604
...                    ...
18283           613.950276
18284           980.976591
18285           469.137365
18286          1196.282697
18287          2472.811495

[4383 rows x 1 columns]
